In [2]:
import json
import numpy as np
import pandas as pd

# Mock JSON data (must have outer braces to be valid JSON)
json_data = '''
{
  "results": {
    "SLSQPSolver": {
      "success": true,
      "message": "Optimization terminated successfully",
      "payload_fraction": 0.030646198517721036,
      "constraint_violation": 0.0,
      "execution_metrics": {
        "iterations": 1,
        "function_evaluations": 3,
        "execution_time": 0.002041339874267578
      },
      "stages": [
        {
          "stage": 1,
          "delta_v": 4650.0,
          "Lambda": 0.2188759008726926
        },
        {
          "stage": 2,
          "delta_v": 4650.0,
          "Lambda": 0.256125802081186
        }
      ]
    },
    "BasinHoppingOptimizer": {
      "success": true,
      "message": "CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH",
      "payload_fraction": 0.009512405918098078,
      "constraint_violation": 0.0,
      "execution_metrics": {
        "iterations": 100,
        "function_evaluations": 0,
        "execution_time": 0.06617045402526855
      },
      "stages": [
        {
          "stage": 1,
          "delta_v": 1209.1873647532418,
          "Lambda": 0.673635367155553
        },
        {
          "stage": 2,
          "delta_v": 8090.812635246759,
          "Lambda": 0.09348299809183877
        }
      ]
    },
    "ParticleSwarmOptimizer": {
      "success": true,
      "message": "PSO optimization completed successfully",
      "payload_fraction": 0.03099855028608968,
      "constraint_violation": 0.0,
      "execution_metrics": {
        "iterations": 31,
        "function_evaluations": 4650,
        "execution_time": 0.7524096965789795
      },
      "stages": [
        {
          "stage": 1,
          "delta_v": 4229.620365437531,
          "Lambda": 0.2511000725784548
        },
        {
          "stage": 2,
          "delta_v": 5070.379634562468,
          "Lambda": 0.2264514025155264
        }
      ]
    },
    "GeneticAlgorithmSolver": {
      "success": true,
      "message": "Optimization completed successfully",
      "payload_fraction": -0.023756279588351883,
      "constraint_violation": 0.0,
      "execution_metrics": {
        "iterations": 200,
        "function_evaluations": 20000,
        "execution_time": 1.3036580085754395
      },
      "stages": [
        {
          "stage": 1,
          "delta_v": 9300.0,
          "Lambda": 0.04790665998283276
        },
        {
          "stage": 2,
          "delta_v": 0.0,
          "Lambda": 1.0
        }
      ]
    },
    "AdaptiveGeneticAlgorithmSolver": {
      "success": true,
      "message": "Optimization completed successfully",
      "payload_fraction": -0.023756279588351883,
      "constraint_violation": 0.0,
      "execution_metrics": {
        "iterations": 200,
        "function_evaluations": 20000,
        "execution_time": 1.533275842666626
      },
      "stages": [
        {
          "stage": 1,
          "delta_v": 9300.0,
          "Lambda": 0.04790665998283276
        },
        {
          "stage": 2,
          "delta_v": 0.0,
          "Lambda": 1.0
        }
      ]
    },
    "DifferentialEvolutionSolver": {
      "success": true,
      "message": "DE optimization completed successfully",
      "payload_fraction": 0.030998550948604833,
      "constraint_violation": 0.0,
      "execution_metrics": {
        "iterations": 110,
        "function_evaluations": 16500,
        "execution_time": 18.436367750167847
      },
      "stages": [
        {
          "stage": 1,
          "delta_v": 4229.046235608008,
          "Lambda": 0.2511471783456773
        },
        {
          "stage": 2,
          "delta_v": 5070.953764391991,
          "Lambda": 0.22641332221589747
        }
      ]
    }
  }
}
'''

# 1. Parse the JSON data
data = json.loads(json_data)

# 2. Define reference constants
g0 = 9.81  # m/s^2
total_delta_v = 9300
isp_stage1 = 312.0
isp_stage2 = 348.0
epsilon1 = 0.07
epsilon2 = 0.08

# 3. Helpers to compute rocket‐equation lambda and 2‐stage payload fraction
def calculate_lambda(delta_v, isp):
    return np.exp(-delta_v / (g0 * isp))

def calculate_payload_fraction(lambda1, lambda2, epsilon1, epsilon2):
    term1 = (lambda1 - epsilon1) / (1 - epsilon1)
    term2 = (lambda2 - epsilon2) / (1 - epsilon2)
    return term1 * term2

# 4. Create a structure to hold comparisons
results = {
    "Solver": [],
    "Stage 1 Delta V": [],
    "Stage 1 Lambda (Computed)": [],
    "Stage 1 Lambda (Documented)": [],
    "Stage 2 Delta V": [],
    "Stage 2 Lambda (Computed)": [],
    "Stage 2 Lambda (Documented)": [],
    "Total Delta V": [],
    "Total Delta V Valid?": [],
    "Payload Fraction (Computed)": [],
    "Payload Fraction (Doc)": [],
    "Difference": []
}

# 5. Optional mapping for better display
solver_display_names = {
    "SLSQPSolver": "SLSQP",
    "BasinHoppingOptimizer": "Basin Hopping",
    "DifferentialEvolutionSolver": "Differential Evolution",
    "ParticleSwarmOptimizer": "Particle Swarm",
    "GeneticAlgorithmSolver": "Genetic Algorithm",
    "AdaptiveGeneticAlgorithmSolver": "Adaptive Genetic Algorithm"
}

# 6. Loop over each solver's data, do computations & store
for solver_key, solver_data in data["results"].items():
    delta_v1 = solver_data["stages"][0]["delta_v"]
    delta_v2 = solver_data["stages"][1]["delta_v"]
    lambda1_doc = solver_data["stages"][0]["Lambda"]
    lambda2_doc = solver_data["stages"][1]["Lambda"]
    payload_fraction_doc = solver_data["payload_fraction"]
    
    # Compute rocket-eq lambdas
    lambda1_computed = calculate_lambda(delta_v1, isp_stage1)
    lambda2_computed = calculate_lambda(delta_v2, isp_stage2)
    
    # 2-stage payload fraction
    payload_fraction_cmp = calculate_payload_fraction(lambda1_computed, lambda2_computed, epsilon1, epsilon2)
    
    total_dv_computed = delta_v1 + delta_v2
    dv_is_valid = np.isclose(total_dv_computed, total_delta_v, rtol=1e-5)
    
    results["Solver"].append(solver_display_names.get(solver_key, solver_key))
    results["Stage 1 Delta V"].append(delta_v1)
    results["Stage 1 Lambda (Computed)"].append(round(lambda1_computed, 5))
    results["Stage 1 Lambda (Documented)"].append(round(lambda1_doc, 5))
    results["Stage 2 Delta V"].append(delta_v2)
    results["Stage 2 Lambda (Computed)"].append(round(lambda2_computed, 5))
    results["Stage 2 Lambda (Documented)"].append(round(lambda2_doc, 5))
    results["Total Delta V"].append(round(total_dv_computed, 2))
    results["Total Delta V Valid?"].append(dv_is_valid)
    results["Payload Fraction (Computed)"].append(round(payload_fraction_cmp, 6))
    results["Payload Fraction (Doc)"].append(round(payload_fraction_doc, 6))
    results["Difference"].append(round(abs(payload_fraction_cmp - payload_fraction_doc), 8))

# 7. Convert to DataFrame for display
df = pd.DataFrame(results)
print("Validation of Solvers\n")
print(df.to_string(index=False))


Validation of Solvers

                    Solver  Stage 1 Delta V  Stage 1 Lambda (Computed)  Stage 1 Lambda (Documented)  Stage 2 Delta V  Stage 2 Lambda (Computed)  Stage 2 Lambda (Documented)  Total Delta V  Total Delta V Valid?  Payload Fraction (Computed)  Payload Fraction (Doc)  Difference
                     SLSQP      4650.000000                    0.21888                      0.21888      4650.000000                    0.25613                      0.25613         9300.0                  True                     0.030646                0.030646         0.0
             Basin Hopping      1209.187365                    0.67364                      0.67364      8090.812635                    0.09348                      0.09348         9300.0                  True                     0.009512                0.009512         0.0
            Particle Swarm      4229.620365                    0.25110                      0.25110      5070.379635                    0.22645         